In [ ]:
#### import library for kinect
import cv2
from pykinect2 import PyKinectV2
from pykinect2 import PyKinectRuntime
import numpy as np
import datetime
import scipy.io 
import timeit

In [ ]:
import argparse
import os
import sys
import time
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.utils.data
from sklearn.metrics import confusion_matrix
from data_loader_jpeg import VideoFolder
from model import ConvColumn
from torchvision.transforms import *
from PIL import Image
gpus = [0,0]
best_prec1 = int(0)


In [ ]:
 # set run output folder
model_name = "jester_conv_example"
start = time.time()
# create model
model = ConvColumn(7)

# multi GPU setting
model = torch.nn.DataParallel(model, device_ids= gpus).cuda()

if os.path.isfile("checkpoint.pth.tar"):
    print("=> loading checkpoint")
    checkpoint = torch.load("checkpoint.pth.tar")
    start_epoch = checkpoint['epoch']
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    print("=> loaded checkpoint (epoch {})"
          .format(checkpoint['epoch']))
else:
    print("=> no checkpoint found ")

cudnn.benchmark = False
print(time.time() - start)
gestures = ['Swiping_Left', 'Swiping_Right', 'Swiping_Down', 'Swiping_Up', 'Zooming_In', 'Zooming_Out', 'Doing_other_things']
colors = [(255,0,0),(0,255,0),(0,0,255),(0,255,255),(255,255,0),(255,0,255),(255,255,255)]

In [ ]:
class GestureBrain():
    
    def __init__(self,  model ):
        self.curr_stream = []
        self.model = model 
        self.counter  = 0 

    def adjust_judge_sequence(self):
        frame_diff = len(self.curr_stream) - 18 
        if  frame_diff == 0 :
            return self.curr_stream
        elif frame_diff > 0 :
            return self.curr_stream[frame_diff:]
        else :
            return self.curr_stream[:1] * abs(frame_diff) + self.curr_stream

    def prepreocess_img (self,  img_array):
        
#        img_array = (img_array / 255.)
#         img_array  = cv2.resize(img_array ,(149,84), interpolation = cv2.INTER_CUBIC)
        img_array = cv2.resize(img_array, (149, 84))
#        img_array = np.array(img_array, dtype=np.float32)
        img_array = Image.fromarray(img_array.astype('uint8'), 'RGB')
        transform = Compose([
        CenterCrop(84),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406],
                  std=[0.229, 0.224, 0.225])
        ])
        
        img_array = transform(img_array)
        return (img_array)
    

    def push_img (self, img):

        img = self.prepreocess_img(img )
        self.curr_stream.append(torch.unsqueeze(img, 0))
        if len( self.curr_stream ) > 18 :      ## limit 18 frames
            self.curr_stream.pop(0)

            
    def regonize(self):
        
        self.curr_stream = self.adjust_judge_sequence()
        data = torch.cat(self.curr_stream)
        data = data.permute(1, 0, 2, 3)
        data = data.unsqueeze(0)
        input = data
        
        input_vars = Variable(input)
        output = model(input_vars)
        _, predicted = torch.max(output.data, 1)
        predicted = predicted.cpu().numpy()
        predicted = predicted[0]
        gestures = ['Swiping_Left', 'Swiping_Right', 'Swiping_Down', 'Swiping_Up', 'Zooming_In', 'Zooming_Out', 'Doing_other_things']
        print('predict_gesture: {}'.format(gestures[predicted]))
        index = predicted
# clear  for 1 action        
#         if index != 6 :
# #             print("cleaning!")
# #             self.curr_stream = []         
        return index 
        


In [ ]:

##### optimize gestures
 
gt = ['Doing_other_things']
gt = gt * 8    ##  number of threshold
b = ['Doing_other_things','Doing_other_things']
c = 'Doing_other_things'


def gestures_opt(act):
    
    gt.append(act)
#     if gt[-1] == gt[-2] == gt[-3] == gt[-4] == gt[-5]== gt[-6] == gt[-7] == gt[-8] == gt[-9] == gt[-10]:
    if gt[-1] == gt[-2] == gt[-3] == gt[-4] == gt[-5]== gt[-6]== gt[-7] == gt[-8] :    
        b.append(gt[-1])
    else:
        b.append('Doing_other_things')
    b.pop(0)
    if b[-1] != b[-2]:
        c = b[-1]
    else:
        c = 'Doing_other_things'
    print(c)
    return c


In [ ]:
file = open("prediction_file.txt","w")
file.close()  

file = open("prediction_file1.txt","w")
file.close()  

# get frame by  Kinect
kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Color | PyKinectV2.FrameSourceTypes_Body)
print(type(kinect))

# Get one color picture frame
cv2.namedWindow('Color Image', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Color Image', int(1920/2), int(1080/2))

frame = None
cnt = 0
x0 = 500
y0 = 500
h = 490/2
w = 650/2
blue_color = (255,0,0)

def draw_rectangle(img, x0, y0, color):
    # ok, at least one is good 
    w = 406
    h = 406
    s1 = (int(x0 - w),int(y0 + h))      #### note: may be  transfer x to y
    s2 = (int(x0 + w),int(y0 + h))
    s3 = (int(x0 + w),int(y0 - h))
    s4 = (int(x0 - w),int(y0 - h))
    cv2.line(img,s1,s2,color,5)
    cv2.line(img,s2,s3,color,5)
    cv2.line(img,s3,s4,color,5)
    cv2.line(img,s4,s1,color,5)

counter = 0 
gb = GestureBrain(model)
while(True):

#     colorFrame = kinect.get_last_color_frame()
#     colorFrame = colorFrame.reshape((1080,1920,4))
#     cv2.imshow("Color Image", colorFrame)
    start = timeit.default_timer()  
    # gb = GestureBrain(model)
    
    if (kinect.has_new_body_frame()): 
        bodies = kinect.get_last_body_frame()
        if bodies is not None: 
            colorFrame = kinect.get_last_color_frame()            
            for i in range(0, kinect.max_body_count):
                body = bodies.bodies[i]      ##### set i =1 ???
                if not body.is_tracked: 
                    continue
                joints = body.joints
                # convert joint coordinates to color space 
                joint_points = kinect.body_joints_to_color_space(joints)     ### joint in color spaces
                joint0 = PyKinectV2.JointType_SpineShoulder
                x0 = joint_points[joint0].x
                y0 = joint_points[joint0].y  
                
                #### convert to RGB frames
#                 colorFrame = kinect.get_last_color_frame()
                colorFrame = colorFrame.reshape((1080,1920,4))
                rgb_frame = colorFrame[:,:,:-1]
                
                
                ######## draw block
                draw_rectangle(colorFrame, x0, y0, colors[i])
                   
                
                ########### boclk frame (note: may be change x,y)
                h1 = 480/2
                w1 = 640/2
                cnt = cnt + 1                        

                    
                roi_frame = rgb_frame[int(y0- h1):int(y0 + h1),int(x0 -w1):int(x0 + w1),:]
                print(np.shape(roi_frame))
                
                counter += 1
#                 time.sleep(0.05)

                
                gb.push_img(roi_frame)
                
#                 if counter != 18 :
#                     continue 
#                 else:
#                     counter = 0 
                
                action = gb.regonize()
                act_name = gestures[action]
                predict_gesture = gestures_opt(act_name)
                
                                
                file = open("prediction_file1.txt", "a")
                file.writelines("%s \n" % (predict_gesture))
                file.close()
                                
                file = open("prediction_file.txt", "a")
                file.writelines("%s \n" % (gestures[action]))
                file.close()              
                
                font = cv2.FONT_HERSHEY_SIMPLEX
#                 cv2.putText(colorFrame,'{}'.format(gestures[action]),(int(x0 - w),int(y0 - h)), font, 2 ,(0,255,0),2,cv2.LINE_AA)
#                cv2.putText(colorFrame,'predict_gesture_per_frame: {}'.format(gestures[action]),(int(80),int(80)), font, 2 ,(255,0,0),2,cv2.LINE_AA)                
                if predict_gesture != 'Doing_other_things': 
                    cv2.putText(colorFrame,'{}'.format(predict_gesture),(int(500),int(300)), font, 3 ,(255,0,255),2,cv2.LINE_AA)
                cv2.imshow("Color Image", colorFrame)
 
                if predict_gesture != 'Doing_other_things': 
                    key = cv2.waitKey(750)
                    time.sleep(0.8)
                    gb.curr_stream = []
                    
                    
                 
                

#                 if action != 6 :
#                      key = cv2.waitKey(150)
#                 path = 'D:\\Hand_Gesture_Recognition_System\\RGB_Output'
#                 fmtname = datetime.datetime.today().strftime("%y%m%d_%H%M%S")
#                cv2.imwrite('{}\\Color_{}_roi_{}.png'.format(path, fmtname, cnt), roi_frame)
                pass
 
    key = cv2.waitKey(20)
    if  key == 27:
        break
    stop = timeit.default_timer()
    fps = 1/(stop-start)
    
    print("Estimated fps : {0}".format(fps))
cv2.destroyAllWindows()
kinect.close()